In [1]:
import numpy as np
import cv2  
import torch
import os

In [3]:
def resize_mask(mask, shape):
    return cv2.resize(mask, (shape[1], shape[0]), interpolation=cv2.INTER_LINEAR)

def convert_to_grayscale(mask):
    if len(mask.shape) == 3 and mask.shape[2] == 3:
        grayscale_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        return grayscale_mask
    return mask

def calculate_iou(mask1, mask2):
    target_shape = max(mask1.shape, mask2.shape, key=lambda x: x[0] * x[1])

    
    resized_mask1 = resize_mask(mask1, target_shape)
    resized_mask2 = resize_mask(mask2, target_shape)
    
    
    intersection = np.logical_and(resized_mask1, resized_mask2).sum()
    union = np.logical_or(resized_mask1, resized_mask2).sum()
    iou = intersection / union if union != 0 else 0
    
    return iou

In [4]:
def load_tensors_from_folder(folder_path):
    tensor_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pth')])
    tensors = {}
    for file in tensor_files:
        tensor_path = os.path.join(folder_path, file)
        tensors[file] = torch.load(tensor_path)
    return tensors

In [12]:
def load_tensors_from_folders(base_folder):
    tensors = {}
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            tensor_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pth')])
            for file in tensor_files:
                tensor_path = os.path.join(folder_path, file)
                tensors[f'{folder_name}.pth'] = torch.load(tensor_path)
    return tensors

In [18]:
tensors_lime_1 = load_tensors_from_folder('../task-2/results/boundary_1/masks/')

In [7]:
print(tensors_lime_1.keys())

dict_keys(['n01443537_goldfish.JPEG.pth', 'n01491361_tiger_shark.JPEG.pth', 'n01608432_kite.JPEG.pth', 'n01616318_vulture.JPEG.pth', 'n01677366_common_iguana.JPEG.pth', 'n02007558_flamingo.JPEG.pth', 'n02018207_American_coot.JPEG.pth', 'n02098286_West_Highland_white_terrier.JPEG.pth', 'n04037443_racer.JPEG.pth', 'n07747607_orange.JPEG.pth'])


In [19]:
tensors_lime_2 = load_tensors_from_folder('../task-2/results/boundary_2/masks/')

In [20]:
tensors_gradcam = load_tensors_from_folders('../task-3/results/gradcam_images/')
print(tensors_gradcam.keys())

dict_keys(['n01608432_kite.JPEG.pth', 'n07747607_orange.JPEG.pth', 'n01616318_vulture.JPEG.pth', 'n02098286_West_Highland_white_terrier.JPEG.pth', 'n01491361_tiger_shark.JPEG.pth', 'n02018207_American_coot.JPEG.pth', 'n02007558_flamingo.JPEG.pth', 'n01677366_common_iguana.JPEG.pth', 'n01443537_goldfish.JPEG.pth', 'n04037443_racer.JPEG.pth'])


In [21]:
keys = tensors_gradcam.keys()

In [22]:
assert tensors_lime_1.keys() == tensors_lime_2.keys() == tensors_gradcam.keys(), "Keys do not match across tensors."


In [23]:
for key in keys:
    mask1 = tensors_lime_1[key].astype('float32')
    mask2 = tensors_gradcam[key].astype('float32')
    mask2 = convert_to_grayscale(mask2)
    iou = calculate_iou(mask1, mask2)
    print(f"IoU for {key}: {iou}")

IoU for n01608432_kite.JPEG.pth: 0.31767099545351785
IoU for n07747607_orange.JPEG.pth: 0.3878355715437891
IoU for n01616318_vulture.JPEG.pth: 0.39001347310308565
IoU for n02098286_West_Highland_white_terrier.JPEG.pth: 0.4457364557145485
IoU for n01491361_tiger_shark.JPEG.pth: 0.36643928881741905
IoU for n02018207_American_coot.JPEG.pth: 0.4086517513231509
IoU for n02007558_flamingo.JPEG.pth: 0.27994839660892
IoU for n01677366_common_iguana.JPEG.pth: 0.289376939335342
IoU for n01443537_goldfish.JPEG.pth: 0.2827821610430306
IoU for n04037443_racer.JPEG.pth: 0.23062286777315227


In [24]:
for key in keys:
    mask1 = tensors_lime_2[key].astype('float32')
    mask2 = tensors_gradcam[key].astype('float32')
    mask2 = convert_to_grayscale(mask2)
    iou = calculate_iou(mask1, mask2)
    print(f"IoU for {key}: {iou}")

IoU for n01608432_kite.JPEG.pth: 0.31767099545351785
IoU for n07747607_orange.JPEG.pth: 0.3457363050964628
IoU for n01616318_vulture.JPEG.pth: 0.3570262780278882
IoU for n02098286_West_Highland_white_terrier.JPEG.pth: 0.3845009668302841
IoU for n01491361_tiger_shark.JPEG.pth: 0.36643928881741905
IoU for n02018207_American_coot.JPEG.pth: 0.3853189643628573
IoU for n02007558_flamingo.JPEG.pth: 0.5342803347280335
IoU for n01677366_common_iguana.JPEG.pth: 0.289376939335342
IoU for n01443537_goldfish.JPEG.pth: 0.2827821610430306
IoU for n04037443_racer.JPEG.pth: 0.2281687239057851
